<a href="https://colab.research.google.com/github/Tndeal/Fashion-recommendation-system/blob/main/Fashion_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## imports
from bs4 import *
import requests
import os
from os import listdir
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import decode_predictions
import numpy as np
import os
!pip install google-search-results
!pip install googledriver
!pip install pyyaml h5py
from serpapi import GoogleSearch
vgg16 = keras.applications.vgg16

from google.colab import drive
drive.mount('/content/drive')

#sets up tensorflow configuration
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

#sort data
!unzip /content/drive/MyDrive/Adatamanipulation/img.zip

In [ ]:
#extracts all possible category names from the list of images
def sortdata(data_file):
  category_list = []
  text_file = open(data_file)
  lines = text_file.readlines()
  for i in range (2, len(lines)):
    split = lines[i].split('/')
    title = split[1]
    if len(title.split('_')) >= 3:
        title = title[0]+'_'+title[-1]
    if title not in category_list:
      category_list.append(title)
  print(len(category_list))
  return category_list
category_list = sortdata('/content/drive/MyDrive/Adatamanipulation/list_category_img.txt')

383


In [ ]:
## MODEL



#sets up training and validation datasets
def setup_datasets(dataset_location):
  imagedatagenerator = keras.preprocessing.image.ImageDataGenerator(
      preprocessing_function = vgg16.preprocess_input,
      horizontal_flip = True,
      validation_split = 0.1)

  train_dataset = imagedatagenerator.flow_from_directory(
      directory = dataset_location,
      target_size = (100, 100),
      classes = category_list,
      batch_size = 32,
      subset = 'training')

  val_dataset = imagedatagenerator.flow_from_directory(
      directory = dataset_location,
      target_size = (100, 100),
      classes = category_list,
      batch_size = 32,
      subset = 'validation')

  return train_dataset, val_dataset

#changes vgg16 model to specific parameters
def alter_model(vgg_model):
  x = keras.layers.Flatten()(vgg_model.output)
  x = keras.layers.Dense(100, activation='relu')(x)
  x = keras.layers.Dense(50, activation='relu')(x)
  predictions = keras.layers.Dense(383, activation='softmax')(x)
  full_model = keras.models.Model(inputs=vgg_model.input, outputs=predictions)
  full_model.summary()
  return full_model


#create training and valdidation datasets
train_dataset, val_dataset = setup_datasets('/content/img')
X_train, y_train = next(train_dataset)


#setup initial vgg16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(100,100,3))
vgg_model.trainable = False
vgg_model.summary()

#alter vgg16 model
full_model = alter_model(vgg_model)
full_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['acc'])

#train model
history = full_model.fit(train_dataset, validation_data = val_dataset, workers=0, epochs=50)


383
Found 11079 images belonging to 383 classes.
Found 1130 images belonging to 383 classes.
58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None,

In [ ]:
full_model.save('/content/drive/MyDrive/my_model.h5')

In [ ]:
## INPUTS
from bs4 import BeautifulSoup
import requests
import os
from os import listdir
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
model = tf.keras.models.load_model('/content/drive/MyDrive/my_model.h5')

## RUN MODEL ON INPUTS

#prepares the images for the model and runs the model to predict categories
def run_model(path):
  img = tf.keras.preprocessing.image.load_img(path, target_size=(100,100))
  subject = tf.keras.preprocessing.image.img_to_array(img)
  subject = preprocess_input(subject)
  subject = np.expand_dims(subject, axis = 0)
  features = model.predict(subject)
  features = np.argmax(features, axis=1)
  features = int(features)
  name = category_list[features]
  return name, img

#returns list of categories of the images
def return_model_run():

  allimages = []
  for i in os.listdir(userfilepath):
    name, image = run_model(os.path.join(userfilepath, i))
    allimages.append(name)
  return allimages


#sorts list of categories by most common
def find_frequencies(category_list):
  frequency = {}

  for name in category_list:
    if name in frequency:
      frequency[name] += 1
    else:
      frequency[name] = 1

  frequency_list = []
  for key, value in frequency.items():
    frequency_list.append([key, value])


  return frequency_list

def merge(left, right):
    i = 0
    j = 0
    sortedlist = []
    while i < len(left) and j < len(right):
        if left[i][1] > right[j][1]:
            sortedlist.append(left[i])
            i += 1
        else:
            sortedlist.append(right[j])
            j += 1

    while j < len(right):
        sortedlist.append(right[j])
        j += 1
    while i< len(left):
        sortedlist.append(left[i])
        i += 1
    return sortedlist

def mergesort(givenlist):
    if len(givenlist) != 1:
        midpoint = len(givenlist) // 2
        left = givenlist[:midpoint]
        right = givenlist[midpoint:]
        left = mergesort(left)
        right = mergesort(right)
        return merge(left, right)
    else:
      return givenlist

#TAKING INPUTS

"""user_file = input('please enter folder name containing images')
userfilepath = os.path.join('/content', user_file)
exists = os.path.exists(userfilepath)
while exists == False:
  check = input('folder does not exist, please enter a different folder name')
  if (os.path.exists(check)) == False:
    exists = False
  else:
    exists = True
"""

#SHARING A GOOGLE DRIVE LINK
url = input("please enter google drive link")
  #url validation
while 'https://drive.google.com/' not in url:
  url = input("please enter google drive link")
print("link accepted")

  #user file validation
user_file = input('please enter file name to be stored under')
userfilepath = os.path.join('/content', user_file)
exists = os.path.exists(userfilepath)
valid = False
while exists == True:
  check = input('file already exists, use this folder? Y/N')
  if check == 'N':
    userfilepath = input('please enter folder name to be stored under')
    exists = os.path.exists(userfilepath)
  else:
    valid = True
    print(f'{userfilepath} will be used as images folder')
    exists = False
if valid == False:
  os.makedirs(userfilepath)
  print(f'{userfilepath} created')

  #download images process

from googledriver import download_folder
download_folder(url, userfilepath)

maxprice = input('please enter max price of clothing')
while maxprice.isdigit() == False:
  print('Please enter a numerical value')
  maxprice = input('please enter max price of clothing')
print('price accepted')


#test model and return image-class pairs
allimages = return_model_run()
print(allimages)
#sort classes into most popular
allimages = find_frequencies(allimages)
sorted = mergesort(allimages)
sorted_list = []
for i in range (0, len(sorted)):
  sorted_list.append(sorted[i][0])



please enter google drive linkhttps://drive.google.com/drive/folders/1EFgLw_qYrTjzmCTf4AoBzjA71UQJK1zD?usp=drive_link
link accepted
please enter file name to be stored underrecommendations
file already exists, use this folder? Y/NY
/content/recommendations will be used as images folder
please enter max price of clothing40
price accepted
1/1 [==============================] - 0s 120ms/step


<ipython-input-8-843f1c7526f8>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features = int(features)


1/1 [==============================] - 0s 119ms/step
['Adios_Jersey', 'Open-Back_Blouse', 'Satin_Joggers']


In [ ]:
## RESULTS

#uses API and parameters to search for the most popular category from the images
#subject to maximum price given
def search_ebay(item):
  params = {
    "engine": "ebay",
    "_nkw": f"{item}",
    "location": "United Kingdom",
    "ebay_domain": "ebay.com",
    "hl": "en",
    "gl": "uk",
    "api_key": "5d4e269b5b82f516868d604bbbfbc27353b4890f26b7403ffb465210d90f8d3e",
    "tbs": f",ppr_min:0,ppr_max:{maxprice}",
    "num": "3"
  }
  search = GoogleSearch(params)
  results = search.get_dict()
  return results

#return ebay search results
for i in range (0, 3):
  item = sorted_list[i].split('_')
  item = ' '.join(item)
  results = search_ebay(item)
#results = pd.DataFrame.from_dict(results)
  for i in range (0, 3):
    print(results['organic_results'][i]['link'])

print(results)

EBAY LINKS

https://www.ebay.com/itm/395546323498?itmmeta=01J3N3103T9PFTQ8GDNY9WET53&hash=item5c1865ea2a:g:xmgAAOSw~4BmnWKw&itmprp=enc%3AAQAJAAAAwIT%2FZskq%2B7jLS8NAovxUWPwxl6fLTsyeQgwt%2FxGswVdfspjaDq5wiwewW8RAuUBdrLlxXiydIHTIDLP1dHzGKEPeA6ucDv4GCko83Fg4DmF949xntMnU1bRZjN61a780K3OvkD%2B8Atha8SgB1JYJSOySC%2BWuTsvwSO3wd7lEQLLPoMnvfpFpBqmlu7RptajS0MoMx5sUcrIkEMBXtspXJpEhBamsinnA%2B77eJpDU9ZHMMYq0KRzslBBEwrlB0sq%2BiQ%3D%3D%7Ctkp%3ABlBMUPyBhKOdZA
https://www.ebay.com/itm/166852633651?itmmeta=01J3N3103T4C8H1GQ9QVPEG59K&hash=item26d9312433:g:GyoAAOSwly9mhYpx&itmprp=enc%3AAQAJAAAA4AzumqSv4XnpWJZu1kNEgQr%2FTtbChzngWgJp3cAI7fwB634EgwrlQgn8IEveNNKrK7rT5IvhgL7mgRr6674gT575BfJet8nqidairRPXzkDtk%2FTJOWqWSPeu8qcI17LF4KuHdguwhfOluHDpIsT6p5XODV0XVt0sRlduDWeOlnGtLw%2FNMYvcSNpZBzs%2Btb%2FuaG%2FxP%2B2bjXMxVqSf1UJ461Kr%2BsYV2hMuFkKnf2SIBQAjFqoBxFIPjhl%2FtYKi6If%2Bo%2FRrozKyw8a1VhEU83MmRbNyWDfiQlpTEY%2FPvGV556PZ%7Ctkp%3ABFBM_IGEo51k
https://www.ebay.com/itm/176484852761?itmmeta=01J3N3103T2FYB8HPBS15QSCBC&h